In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime as tm
import datetime
import seaborn as sns
t = time.time()

In [2]:
### converts varying datatypes into unix time for easier computation
def convert2unix(time):
    ### for string format
    if type(time) == type('hey'):
        time = (tm.strptime(time,'%Y-%m-%dT%H:%M:%S') - tm(1970,1,1,0,0,0)).total_seconds()
        #time = tm.strptime(time,'%m/%d/%y %H:%M:%S %p').timestamp()
    ### for datetime format
    elif type(time) == tm:
        time = time.timestamp()
        print('tm')
    ### this is already in unix time
    elif type(time) == float:
        #print(time)
        pass
    ### for pandas format
    elif type(time) == pd._libs.tslibs.timestamps.Timestamp:
        time = time.timestamp()
    ### notifies the user that the format is not recognized
    else:
        print('NONE OF THE ABOVE')
        print(type(time))
    return time


In [3]:
def find_zero(lst, lastfail):
    newlst = []
    for item in lst:
        if item >= 0 and item >= lastfail:
            newlst.append(item - lastfail) 
    return(newlst)

In [4]:

### $ Filename imported by field engineer
filename = 'C:/Users/sambe/Desktop/Chevron Project/chevron_test_template.xlsx'
### list of all possible motor variable inputs
allvars = ['Inlet Flowrate',
 'Inlet Pressure',
 'Outlet Pressure',
 'Inlet Temperature',
 'Column Level',
 'STD API',
 'Pump Driver Vibration',
 'Pump Vibration',
 'Sister Pump Driver Vibration',
 'Sister Pump Vibration']
### $ Convert to pandas data frame
df = pd.read_excel(filename, sheet_name = 'Time-Series Data')
print(f'Elapsed time is {(time.time() - t)/60} minutes')
### grab columns from input dataframe
a = df.columns

### $ Get the number of rows and columns in the file (# of covariates and sampling will probably be different)
number_rows = len(df.index)
number_cols = len(df.columns)


Elapsed time is 1.56085201104482 minutes


In [5]:
### formatting
df.columns = df.iloc[0].fillna('Unnamed')
df.drop(0, inplace = True)
df.reset_index(drop = True, inplace = True)
df

,Pen Name:,16FI226,Units: MBPD,Unnamed,Pen Name:,16pi2115,Units: PSIG,Unnamed,Pen Name:,16pi2115,...,16ve1133,Units: IN/SEC,Unnamed,EMPTY 1,EMPTY 2,EMPTY 3,Unnamed,EMPTY 1,EMPTY 2,EMPTY 3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2038.28,2018-02-20T15:01:30,GOOD,NaN,44.0469,2018-02-20T15:01:30,GOOD,NaN,5574.77,2018-02-20T15:01:30,...,2018-02-20T15:01:30,GOOD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2039.22,2018-02-20T15:01:59,GOOD,NaN,44.0703,2018-02-20T15:01:59,GOOD,NaN,5565.62,2018-02-20T15:01:59,...,2018-02-20T15:01:59,GOOD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2039.06,2018-02-20T15:02:29,GOOD,NaN,44.1953,2018-02-20T15:02:29,GOOD,NaN,5623.01,2018-02-20T15:02:29,...,2018-02-20T15:02:58,GOOD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2039.22,2018-02-20T15:03:29,GOOD,NaN,44.0469,2018-02-20T15:03:29,GOOD,NaN,5582.38,2018-02-20T15:02:58,...,2018-02-20T15:05:26,BAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

### $ Create empty data frames with the three points per covariate (value, times, good/bad)
variables = pd.DataFrame([])
times = pd.DataFrame()
goodbad = pd.DataFrame()
### $ Sort out whole excel file to place the column in the right data frame (value, times, good/bad)
### $ Relabeled with Variable i names
### $ Iterates over the number of columns
t = time.time()
missingcols = []
counter = 0
for i in range(number_cols):
    ### $ Puts column title into str
    str = df.columns[i]
    ### $ if searches for a title that starts with 'Pen Name'
    if str.startswith('Pen Name'):
        ### $ takes variables from column and puts in separate 'variables' dataframe
        s = df.iloc[1:,i]
        variables = pd.concat([variables,s.rename('Value')], axis = 1)
    ### $ elif searches for a title that starts with 'Units'
    elif str.startswith('Units'):
        ### $ takes variables from column and puts in separate 'goodbad' dataframe
        s = df.iloc[1:,i]
        goodbad = pd.concat([goodbad,s.rename('GOOD/BAD')],axis = 1)
    ### $ This is for the blank columns in between variable columns
    elif str.startswith('Unnamed'):
        ### $ Skips over empty columns
        continue
    elif str.startswith('EMPTY 1'):
        variables = pd.concat([variables,pd.DataFrame([np.nan])],axis = 1)
    elif str.startswith('EMPTY 2'):
        times = pd.concat([times,pd.DataFrame([np.nan])],axis = 1)
        missingcols.append(int((i-1)/4))
    elif str.startswith('EMPTY 3'):
        goodbad = pd.concat([goodbad,pd.DataFrame([np.nan])],axis = 1)
    ### $ This is an else because the pen names vary and will be different for every pump/variable
    else:
        ### $ takes variables from column and puts in separate 'times' dataframe
        s = df.iloc[1:,i].apply(convert2unix)
        times = pd.concat([times,s.rename('Time')], axis = 1)
print(f'Elapsed time is {(time.time() - t)/60} minutes')
### relabels times, goodbad, and variables to have the same column names
times.columns = allvars
goodbad.columns = allvars
variables.columns = allvars

Elapsed time is 0.5453181465466818 minutes


In [7]:
### formatting
times.drop(0,inplace = True)
times.reset_index(drop = True, inplace = True)
variables.drop(0,inplace = True)
variables.reset_index(drop = True, inplace = True)
goodbad.drop(0,inplace = True)
goodbad.reset_index(drop = True, inplace = True)

In [8]:
### finds the available vairables for that motor (i.e. those variables that were input into the program)
availvars = []
for i in range(len(times.columns)):
    if i in missingcols:
        pass
    else:
        availvars.append(times.columns[i])

In [9]:
availvars

['Inlet Flowrate',
 'Inlet Pressure',
 'Outlet Pressure',
 'Pump Driver Vibration',
 'Pump Vibration']

In [10]:
### selects only the columns with variables in them
times = times[availvars]
goodbad = goodbad[availvars]
variables = variables[availvars]

In [11]:
### formatting for later
times = times.append(pd.Series([], dtype = np.float64),ignore_index = True)

In [12]:
### $ interpolates for BAD Values
### $ Excludes last row because you can't interpolate a machine being on or off
t = time.time()
for j in range(len(variables.columns)):
    for i in range(len(variables.iloc[:,j].dropna())):
        str = goodbad.iloc[i,j]
        ### $ If the value is bad and the value has no prior value to interpolate from, just use next good value
        ### $ Needs improvement: could back interpolate and take care of case in which there are consecutive bad values     
        if str.startswith('B'):# and i == 0:
            variables.iloc[i,j] = variables.iloc[i-1,j]
            goodbad.iloc[i,j] = 'OK'
        ### $ Else if bad, then interpolate between the two good points
#        elif str.startswith('B'):
#            t1 = times.iloc[i-1,j]
#            t2 = times.iloc[i,j]
#            t3 = times.iloc[i+1,j]
#            a = variables.iloc[i-1,j]
#            c = variables.iloc[i+1,j]
#            variables.iloc[i,j] = ((t2-t1)/(t3-t1))*(c-a)+a
#            goodbad.iloc[i,j] = 'INT'
print(f'Elapsed time is {(time.time() - t)/60} minutes')

Elapsed time is 0.8584701498349507 minutes


In [13]:
### $ Align times via binning
### $ kappa designates the grouping size
### $ Any timestamp outside of the bin will be group later



### $ Sets to dataframes: windows and shutters. windows is the number row that is being looked at for that variable. shutters is end of the length of that variable
### $ column titles designate which variable is being investigated
windows = pd.DataFrame()
shutters = pd.DataFrame()
for j in range(len(variables.columns)):
    w = pd.DataFrame([0], columns = [f'{j}'])
    windows = pd.concat([windows, w], axis = 1)
    temp = variables.iloc[:,j]
    s = pd.DataFrame([len(temp)], columns = [f'{j}'])
    shutters = pd.concat([shutters, s], axis = 1)
    
binsize = 60*2#seconds

### $ abstime is the merged and grouped times of all the variables
abstime = pd.DataFrame([])
### $ vars is the merged and grouped variables
vars = pd.DataFrame([], columns = allvars)

In [14]:
############# binsize for binning in seconds
binsize = 60*2
#############
timestart = times.dropna(thresh = 1).min().min()
timenow = timestart
timeend = times.dropna(thresh = 1).max().max()
t = time.time()

abstime = []
vars = []
### Flag denotes if the timenow is less than or equal to timeend
Flag = 1
while Flag:
    ### vlist is list of variables in that timebin
    vlist = []
    ### below iterates over columns
    for j in range(len(variables.columns)):
        ### Flag1 denotes if the end of the available times in the column has been reached
        Flag1 = 1
        ### v is the list of variables that fall within the time bin (say three flowrates occur within that two minute timebin)
        v = []
        while Flag1:
            ###
            timecomp = times.iloc[windows.iloc[0,j],j]
            if timecomp >= timenow and timecomp < timenow + binsize:
                v.append(variables.iloc[windows.iloc[0,j],j])
                windows.iloc[0,j] += 1 
            elif timecomp >= timenow + binsize:
                Flag1 = 0
            elif np.isnan(times.iloc[windows.iloc[0,j],j]):
                Flag1 = 0   

            else:
                print('HELP')
        if len(v) == 0:
            v = np.nan
        else:
            v = sum(v)/float(len(v)) 
        vlist.append(v)

        
    abstime.append(timenow)
    vars.append(vlist)
    timenow += binsize
    if timenow > timeend:
        Flag = 0
print(f'Elapsed time is {(time.time() - t)/60} minutes')

Elapsed time is 9.202732316652934 minutes


In [15]:
abstime = pd.DataFrame(abstime, columns = ['Time'])

In [16]:
vars = pd.DataFrame(vars, columns = availvars)

In [17]:
for j in range(len(vars.columns)):
    for i in range(len(vars)):
        if np.isnan(vars.iloc[i,j]) and i == 0:
            vars.iloc[i,j] = vars.iloc[i+1,j]
        elif np.isnan(vars.iloc[i,j]):
            vars.iloc[i,j] = vars.iloc[i-1,j]


In [18]:
vars

,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration
0,2038.945312,44.089844,5589.695313,0.072129,0.022539
1,2040.156250,44.150391,5585.048829,0.073216,0.022109
2,2040.937500,44.190104,5592.539063,0.074336,0.023398
3,2042.578125,44.185547,5595.966797,0.074385,0.022520
4,2042.395833,44.158203,5592.539063,0.073105,0.022721
...,...,...,...,...,...
525597,1727.500000,40.437500,7743.125000,0.121094,0.021367
525598,1727.656250,40.257812,7718.242188,0.111328,0.021367
525599,1728.593750,40.375000,7781.210938,0.105977,0.024687
525600,1726.093750,40.171875,7798.476563,0.111094,0.024687


In [21]:
availvarsplustime = ['Time']
for s in availvars:
    availvarsplustime.append(s)


In [22]:


fixed = pd.concat([abstime,vars], axis = 1)
fixed.columns = availvarsplustime
fixed.dropna(thresh = 2, inplace = True)



fixed.to_excel('interpolatedvar.xlsx')


In [23]:
fixed

,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration
0,1.519139e+09,2038.945312,44.089844,5589.695313,0.072129,0.022539
1,1.519139e+09,2040.156250,44.150391,5585.048829,0.073216,0.022109
2,1.519139e+09,2040.937500,44.190104,5592.539063,0.074336,0.023398
3,1.519139e+09,2042.578125,44.185547,5595.966797,0.074385,0.022520
4,1.519139e+09,2042.395833,44.158203,5592.539063,0.073105,0.022721
...,...,...,...,...,...,...
525597,1.582211e+09,1727.500000,40.437500,7743.125000,0.121094,0.021367
525598,1.582211e+09,1727.656250,40.257812,7718.242188,0.111328,0.021367
525599,1.582211e+09,1728.593750,40.375000,7781.210938,0.105977,0.024687
525600,1.582211e+09,1726.093750,40.171875,7798.476563,0.111094,0.024687


In [24]:

filename = 'C:/Users/sambe/Desktop/Chevron Project/chevron_test_template.xlsx'
faildata = pd.read_excel(filename, sheet_name = 'Failure Mode & Date')

In [25]:
faildata

,Date,Mode,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,2018-09-24,Valve_Alignment,NaN,2018-02-20 15:04:00,Valve_Alignment,NaN,NaN,NaN,2018-09-24,Valve_Alignment
1,2019-03-08,Cracked_Seal,NaN,2019-03-08 00:00:00,Cracked_Seal,NaN,NaN,NaN,2019-03-08,Cracked_Seal
2,2019-05-16,Broken_Impeller,NaN,2019-05-16 00:00:00,Broken_Impeller,NaN,NaN,NaN,2019-05-16,Broken_Impeller
3,2019-06-03,Broken_Valve,NaN,2019-06-03 00:00:00,Broken_Valve,NaN,NaN,NaN,2019-06-03,Broken_Valve
4,2019-08-05,Leaking_Valve,NaN,2019-08-05 00:00:00,Leaking_Valve,NaN,NaN,NaN,2019-08-05,Leaking_Valve
5,2019-09-28,Motor_Distorted,NaN,2020-02-20 15:02:00,Valve_Alignment,NaN,NaN,NaN,2019-09-28,Motor_Distorted


In [26]:
faildata['Date'] = faildata['Date'].apply(convert2unix)

In [27]:
faildata.sort_values(by = ['Date'], inplace = True)

In [28]:
allfailmodes = ['Motor_Distorted','Broken_Valve','Valve_Alignment','Broken_Impeller','Leaking_Valve', 'Cracked_Seal']
allfailmodeswindows = [60*60*24*2,60*60*24*2,60*60*24*2,60*60*24*2,60*60*24*2,60*60*24*2]

In [29]:
failmodeswindows = pd.DataFrame([allfailmodeswindows], columns = allfailmodes)

In [30]:
failmodeswindows

,Motor_Distorted,Broken_Valve,Valve_Alignment,Broken_Impeller,Leaking_Valve,Cracked_Seal
0,172800,172800,172800,172800,172800,172800


In [31]:
list = []
for mode in faildata['Mode']:
    list.append(failmodeswindows[mode][0])

In [32]:
faildata['Time Window'] = list

In [33]:
faildata.reset_index(drop = True, inplace = True)

In [34]:
faildata

,Date,Mode,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Time Window
0,1.537747e+09,Valve_Alignment,NaN,2018-02-20 15:04:00,Valve_Alignment,NaN,NaN,NaN,2018-09-24,Valve_Alignment,172800
1,1.552003e+09,Cracked_Seal,NaN,2019-03-08 00:00:00,Cracked_Seal,NaN,NaN,NaN,2019-03-08,Cracked_Seal,172800
2,1.557965e+09,Broken_Impeller,NaN,2019-05-16 00:00:00,Broken_Impeller,NaN,NaN,NaN,2019-05-16,Broken_Impeller,172800
3,1.559520e+09,Broken_Valve,NaN,2019-06-03 00:00:00,Broken_Valve,NaN,NaN,NaN,2019-06-03,Broken_Valve,172800
4,1.564963e+09,Leaking_Valve,NaN,2019-08-05 00:00:00,Leaking_Valve,NaN,NaN,NaN,2019-08-05,Leaking_Valve,172800
5,1.569629e+09,Motor_Distorted,NaN,2020-02-20 15:02:00,Valve_Alignment,NaN,NaN,NaN,2019-09-28,Motor_Distorted,172800


In [35]:
list = []
for faildate in faildata['Date']:
    faildiff = faildate - faildata['Date']
    Flag = 0
    for diff in faildiff:
        if diff > 0 and diff < faildata.loc[faildata['Date'] == faildate]['Time Window'].values:
            x = diff
            Flag = 1
    if Flag == 1:
        list.append(x)
    else:
        list.append(faildata.loc[faildata['Date'] == faildate]['Time Window'].values[0])
        

In [36]:
faildata.drop(axis = 1,columns = 'Time Window', inplace = True)

In [37]:
faildata['Time Window'] = list

In [38]:
faildata

,Date,Mode,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Time Window
0,1.537747e+09,Valve_Alignment,NaN,2018-02-20 15:04:00,Valve_Alignment,NaN,NaN,NaN,2018-09-24,Valve_Alignment,172800
1,1.552003e+09,Cracked_Seal,NaN,2019-03-08 00:00:00,Cracked_Seal,NaN,NaN,NaN,2019-03-08,Cracked_Seal,172800
2,1.557965e+09,Broken_Impeller,NaN,2019-05-16 00:00:00,Broken_Impeller,NaN,NaN,NaN,2019-05-16,Broken_Impeller,172800
3,1.559520e+09,Broken_Valve,NaN,2019-06-03 00:00:00,Broken_Valve,NaN,NaN,NaN,2019-06-03,Broken_Valve,172800
4,1.564963e+09,Leaking_Valve,NaN,2019-08-05 00:00:00,Leaking_Valve,NaN,NaN,NaN,2019-08-05,Leaking_Valve,172800
5,1.569629e+09,Motor_Distorted,NaN,2020-02-20 15:02:00,Valve_Alignment,NaN,NaN,NaN,2019-09-28,Motor_Distorted,172800


In [39]:
list = []
#####
minvib = 0.05
#####
vibdiff = fixed['Pump Driver Vibration'] - minvib
for item in vibdiff:
    if item > 0:
        list.append('On')
    else:
        list.append('Off')
fixed['Status'] = list

fixed = fixed[fixed['Status'] == 'On']

fixed.reset_index(drop = True, inplace = True)

In [40]:
fixed

,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration,Status
0,1.519139e+09,2038.945312,44.089844,5589.695313,0.072129,0.022539,On
1,1.519139e+09,2040.156250,44.150391,5585.048829,0.073216,0.022109,On
2,1.519139e+09,2040.937500,44.190104,5592.539063,0.074336,0.023398,On
3,1.519139e+09,2042.578125,44.185547,5595.966797,0.074385,0.022520,On
4,1.519139e+09,2042.395833,44.158203,5592.539063,0.073105,0.022721,On
...,...,...,...,...,...,...,...
301368,1.582211e+09,1727.500000,40.437500,7743.125000,0.121094,0.021367,On
301369,1.582211e+09,1727.656250,40.257812,7718.242188,0.111328,0.021367,On
301370,1.582211e+09,1728.593750,40.375000,7781.210938,0.105977,0.024687,On
301371,1.582211e+09,1726.093750,40.171875,7798.476563,0.111094,0.024687,On


In [41]:
onoffint = 60*60

off = pd.DataFrame(fixed[fixed['Time'].diff(periods = -1) < -onoffint])
offindex = off.index
off.reset_index(drop = True, inplace = True)
off['Time'] = off['Time'] - timestart

on = pd.DataFrame(fixed[fixed['Time'].diff(periods = 1) > onoffint])
onindex = on.index
on.reset_index(drop = True, inplace = True)
on['Time'] = on['Time'] - timestart

onindex

Int64Index([    86,   2988,   3071,   7707,   9307,  11123,  14041,  18486,
             20169,  26871,
            ...
            293272, 293331, 293402, 293514, 293735, 295477, 295773, 295953,
            301134, 301288],
           dtype='int64', length=827)

In [42]:
### Fix to time on
timestart = fixed['Time'].min()
timestart

timeshifted = fixed['Time'] - timestart

timeshifted

timeoff = on['Time'] - off['Time']

timeoff

timeoff.index = onindex

fixedlist = timeshifted.tolist()

for index in fixed.index:
    if index in timeoff.index:
        fixedlist[index:] -= timeoff[index]

timeonlist = pd.DataFrame(fixedlist,columns = ['Time On'])

timeonlist


fixed = pd.concat([timeonlist,fixed], axis = 1)

In [43]:
fixed

,Time On,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration,Status
0,0.0,1.519139e+09,2038.945312,44.089844,5589.695313,0.072129,0.022539,On
1,120.0,1.519139e+09,2040.156250,44.150391,5585.048829,0.073216,0.022109,On
2,240.0,1.519139e+09,2040.937500,44.190104,5592.539063,0.074336,0.023398,On
3,360.0,1.519139e+09,2042.578125,44.185547,5595.966797,0.074385,0.022520,On
4,480.0,1.519139e+09,2042.395833,44.158203,5592.539063,0.073105,0.022721,On
...,...,...,...,...,...,...,...,...
301368,37553160.0,1.582211e+09,1727.500000,40.437500,7743.125000,0.121094,0.021367,On
301369,37553280.0,1.582211e+09,1727.656250,40.257812,7718.242188,0.111328,0.021367,On
301370,37553400.0,1.582211e+09,1728.593750,40.375000,7781.210938,0.105977,0.024687,On
301371,37553520.0,1.582211e+09,1726.093750,40.171875,7798.476563,0.111094,0.024687,On


In [49]:
labeled = pd.DataFrame([], columns = availvarsplustime)

In [50]:
for mode in allfailmodes:
    modedates = faildata[faildata['Mode'] == mode]['Date']
    for failuredate in modedates:
        window = faildata[(faildata['Mode'] == mode) & (faildata['Date'] == failuredate)]['Time Window'].values[0]
        timeonfail = fixed.loc[(fixed['Time'] <= failuredate)]['Time On'].max()
        z = pd.DataFrame(fixed.loc[(fixed['Time'] <= failuredate) & (fixed['Time On'] > timeonfail - window)])
        z['Condition'] = mode
        labeled = pd.concat([labeled,z])

In [51]:
labeled

,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration,Time On,Status,Condition
209524,1.567724e+09,45.78125,45.468750,-0.507812,0.052148,0.004375,26356680.0,On,Motor_Distorted
209525,1.567725e+09,45.46875,45.429688,0.000000,0.051094,0.004375,26356920.0,On,Motor_Distorted
209526,1.567725e+09,45.62500,45.414062,-0.507812,0.051406,0.004375,26357160.0,On,Motor_Distorted
209527,1.567725e+09,45.78125,45.523438,-0.507812,0.060352,0.004375,26357280.0,On,Motor_Distorted
209528,1.567807e+09,45.78125,45.898438,211.757812,0.050625,0.004453,26357280.0,On,Motor_Distorted
...,...,...,...,...,...,...,...,...,...
189587,1.552003e+09,1807.81250,46.960938,5203.554688,0.130781,0.024258,23465280.0,On,Cracked_Seal
189588,1.552003e+09,1810.00000,46.914062,6187.695313,0.141680,0.024258,23465400.0,On,Cracked_Seal
189589,1.552003e+09,1810.00000,46.953125,4991.796875,0.126992,0.020273,23465520.0,On,Cracked_Seal
189590,1.552003e+09,1807.96875,46.882812,5946.484375,0.136445,0.020273,23465640.0,On,Cracked_Seal


In [52]:
window

172800

In [53]:
labeled.sort_index(inplace = True)

In [54]:
labeled

,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration,Time On,Status,Condition
102122,1.537574e+09,1821.71875,43.468750,6571.093750,0.101562,0.030352,12585360.0,On,Valve_Alignment
102123,1.537575e+09,1820.00000,43.515625,6595.468750,0.090625,0.031641,12585480.0,On,Valve_Alignment
102124,1.537575e+09,1819.06250,43.437500,6701.093750,0.076992,0.035195,12585600.0,On,Valve_Alignment
102125,1.537575e+09,1819.53125,43.343750,6655.390625,0.077461,0.038281,12585720.0,On,Valve_Alignment
102126,1.537575e+09,1819.84375,43.359375,6616.289063,0.074375,0.035586,12585840.0,On,Valve_Alignment
...,...,...,...,...,...,...,...,...,...
210173,1.569626e+09,44.37500,45.273438,132.031250,0.052734,0.003047,26526120.0,On,Motor_Distorted
210174,1.569626e+09,44.06250,45.109375,-1.015625,0.067812,0.003047,26526240.0,On,Motor_Distorted
210175,1.569626e+09,44.37500,45.140625,10.664062,0.056016,0.001797,26526480.0,On,Motor_Distorted
210176,1.569629e+09,45.46875,45.406250,-1.015625,0.052109,0.002852,26529120.0,On,Motor_Distorted


In [55]:
list = []
for index1 in fixed.index:
    if index1  not in labeled.index:
        list.append(index1)
        

In [56]:
z = pd.DataFrame(fixed.loc[list,:])
z['Condition'] = 'Normal'
labeled = pd.concat([labeled,z])

In [57]:
labeled.sort_index(inplace = True)

In [58]:
labeled['Condition'].value_counts()

Normal             294187
Valve_Alignment      1440
Cracked_Seal         1440
Broken_Impeller      1363
Broken_Valve         1323
Leaking_Valve         966
Motor_Distorted       654
Name: Condition, dtype: int64

labeled.to_excel('HEYYY.xlsx')

In [59]:
del fixed

list = []
#####
minvib = 0.05
#####
vibdiff = labeled['Pump Driver Vibration'] - minvib
for item in vibdiff:
    if item > 0:
        list.append('On')
    else:
        list.append('Off')
labeled['Status'] = list

labeled = labeled[labeled['Status'] == 'On']

labeled.reset_index(drop = True, inplace = True)

len(labeled)

In [60]:
allfailmodescols = []
allfailtimescols = []
for mode in allfailmodes:
    allfailmodescols.append(f'{mode} Failure')
    allfailtimescols.append(f'{mode} TT Failure')

In [61]:
allfailtimescols

['Motor_Distorted TT Failure',
 'Broken_Valve TT Failure',
 'Valve_Alignment TT Failure',
 'Broken_Impeller TT Failure',
 'Leaking_Valve TT Failure',
 'Cracked_Seal TT Failure']

In [62]:
def pos(lst):
    return [x for x in lst if x > 0] or None

timeend = labeled['Time'].max()
timestart = labeled['Time'].min()
ttime = timeend-timestart
mainlist = []
for mode in allfailmodes:

    list = []
    check = []
    m = faildata[faildata['Mode'] == mode]['Date']
    lastfailtime = 0
    for timefail in m:
        print(mode,timefail)
        Flag1 = 1
        timediff = timefail - labeled['Time']
        truetimefail = timefail - min(pos(timediff)) 
        for time in labeled['Time']:
            if Flag1:
                if time != truetimefail and time > lastfailtime:
                    list.append(0)
                elif time <= lastfailtime:
                    pass
                else:
                    print('sweet')
                    list.append(1)
                    Flag1 = 0
                    lastfailtime = time

#    for timefail in m:
#        print(mode,timefail)
#        Flag1 = 1
#        timediff = timefail - labeled['Time']
#        truetimefail = timefail - min(pos(timediff)) 
#
#        while Flag1:
#            for time in labeled['Time']:
#                if time != truetimefail:
#                    list.append(0)
#                else:
#                    print('sweet')
#                    list.append(1)
#                    Flag1 = 0
                

    if list == []:
        for time in labeled['Time']:
            list.append(0)
#            check.append()
    mainlist.append(list)
    
#for time in labeled['Time']:
 #   if time != 

mainlist = pd.DataFrame(mainlist)

mainlist = mainlist.transpose()

mainlist.columns = allfailmodescols

mainlist

labeled = pd.concat([labeled,mainlist],axis = 1)

labeled

Motor_Distorted 1569628800.0
sweet
Broken_Valve 1559520000.0
sweet
Valve_Alignment 1537747200.0
sweet
Broken_Impeller 1557964800.0
sweet
Leaking_Valve 1564963200.0
sweet
Cracked_Seal 1552003200.0
sweet


,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration,Time On,Status,Condition,Motor_Distorted Failure,Broken_Valve Failure,Valve_Alignment Failure,Broken_Impeller Failure,Leaking_Valve Failure,Cracked_Seal Failure
0,1.519139e+09,2038.945312,44.089844,5589.695313,0.072129,0.022539,0.0,On,Normal,0.0,0.0,0.0,0.0,0.0,0.0
1,1.519139e+09,2040.156250,44.150391,5585.048829,0.073216,0.022109,120.0,On,Normal,0.0,0.0,0.0,0.0,0.0,0.0
2,1.519139e+09,2040.937500,44.190104,5592.539063,0.074336,0.023398,240.0,On,Normal,0.0,0.0,0.0,0.0,0.0,0.0
3,1.519139e+09,2042.578125,44.185547,5595.966797,0.074385,0.022520,360.0,On,Normal,0.0,0.0,0.0,0.0,0.0,0.0
4,1.519139e+09,2042.395833,44.158203,5592.539063,0.073105,0.022721,480.0,On,Normal,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301368,1.582211e+09,1727.500000,40.437500,7743.125000,0.121094,0.021367,37553160.0,On,Normal,NaN,NaN,NaN,NaN,NaN,NaN
301369,1.582211e+09,1727.656250,40.257812,7718.242188,0.111328,0.021367,37553280.0,On,Normal,NaN,NaN,NaN,NaN,NaN,NaN
301370,1.582211e+09,1728.593750,40.375000,7781.210938,0.105977,0.024687,37553400.0,On,Normal,NaN,NaN,NaN,NaN,NaN,NaN
301371,1.582211e+09,1726.093750,40.171875,7798.476563,0.111094,0.024687,37553520.0,On,Normal,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
mainlist

,Motor_Distorted Failure,Broken_Valve Failure,Valve_Alignment Failure,Broken_Impeller Failure,Leaking_Valve Failure,Cracked_Seal Failure
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
210173,0.0,NaN,NaN,NaN,NaN,NaN
210174,0.0,NaN,NaN,NaN,NaN,NaN
210175,0.0,NaN,NaN,NaN,NaN,NaN
210176,0.0,NaN,NaN,NaN,NaN,NaN


onoffint = 60*60

off = pd.DataFrame(labeled[labeled['Time'].diff(periods = -1) < -onoffint])
offindex = off.index
off.reset_index(drop = True, inplace = True)
off['Time'] = off['Time'] - timestart

on = pd.DataFrame(labeled[labeled['Time'].diff(periods = 1) > onoffint])
onindex = on.index
on.reset_index(drop = True, inplace = True)
on['Time'] = on['Time'] - timestart

onindex

### Fix to time on
timestart = labeled['Time'].min()
timestart

a = labeled['Time'] - timestart

a

timeoff = on['Time'] - off['Time']

timeoff

timeoff.index = onindex

labeledlist = a.tolist()

for index in labeled.index:
    if index in timeoff.index:
        labeledlist[index:] -= timeoff[index]

timeonlist = pd.DataFrame(labeledlist,columns = ['Time On'])

timeonlist

labeled.drop(columns = 'Time', inplace = True)

labeled = pd.concat([timeonlist,labeled], axis = 1)

labeled.to_excel('Cleaned_Shit.xlsx')

labeled['Valve_Alignment Failure'].value_counts()

labeled

def find_zero(lst, lastfail):
    newlst = []
    for item in lst:
        if item >= 0 and item >= lastfail:
            newlst.append(item - lastfail) 
    return(newlst)

In [64]:
allttfaillist = []
for mode in allfailmodes:
#mode = 'Motor_Distorted'
    ttfaillist = []
    timelastfail = 0
    for fail in labeled.loc[labeled[f'{mode} Failure'] == 1]['Time On']:
        a = (fail - labeled['Time On']).tolist()
        ttfaillist += find_zero(a,timelastfail)#.append(find_zero(a,timelastfail))
        timelastfail = fail

    if ttfaillist == []:  
        y = pd.DataFrame(index = np.arange(len(labeled)), columns = ['0'])
        y = np.array(y.values).flatten().tolist()
        allttfaillist.append(y)
    else:
        allttfaillist.append(ttfaillist)

In [65]:
allttfail = pd.DataFrame(allttfaillist)

In [66]:
allttfail = allttfail.transpose()

In [67]:
allttfail.columns = allfailtimescols

In [68]:
allttfail

,Motor_Distorted TT Failure,Broken_Valve TT Failure,Valve_Alignment TT Failure,Broken_Impeller TT Failure,Leaking_Valve TT Failure,Cracked_Seal TT Failure
0,26529240.0,24136200.0,12758040.0,23948880.0,26148840.0,23465760.0
1,26529120.0,24136080.0,12757920.0,23948760.0,26148720.0,23465640.0
2,26529000.0,24135960.0,12757800.0,23948640.0,26148600.0,23465520.0
3,26528880.0,24135840.0,12757680.0,23948520.0,26148480.0,23465400.0
4,26528760.0,24135720.0,12757560.0,23948400.0,26148360.0,23465280.0
...,...,...,...,...,...,...
210173,3120.0,NaN,NaN,NaN,NaN,NaN
210174,3000.0,NaN,NaN,NaN,NaN,NaN
210175,2760.0,NaN,NaN,NaN,NaN,NaN
210176,120.0,NaN,NaN,NaN,NaN,NaN


In [69]:
labeledplustt = pd.concat([labeled, allttfail], axis = 1)

In [70]:
labeledplustt

,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Pump Driver Vibration,Pump Vibration,Time On,Status,Condition,Motor_Distorted Failure,...,Valve_Alignment Failure,Broken_Impeller Failure,Leaking_Valve Failure,Cracked_Seal Failure,Motor_Distorted TT Failure,Broken_Valve TT Failure,Valve_Alignment TT Failure,Broken_Impeller TT Failure,Leaking_Valve TT Failure,Cracked_Seal TT Failure
0,1.519139e+09,2038.945312,44.089844,5589.695313,0.072129,0.022539,0.0,On,Normal,0.0,...,0.0,0.0,0.0,0.0,26529240.0,24136200.0,12758040.0,23948880.0,26148840.0,23465760.0
1,1.519139e+09,2040.156250,44.150391,5585.048829,0.073216,0.022109,120.0,On,Normal,0.0,...,0.0,0.0,0.0,0.0,26529120.0,24136080.0,12757920.0,23948760.0,26148720.0,23465640.0
2,1.519139e+09,2040.937500,44.190104,5592.539063,0.074336,0.023398,240.0,On,Normal,0.0,...,0.0,0.0,0.0,0.0,26529000.0,24135960.0,12757800.0,23948640.0,26148600.0,23465520.0
3,1.519139e+09,2042.578125,44.185547,5595.966797,0.074385,0.022520,360.0,On,Normal,0.0,...,0.0,0.0,0.0,0.0,26528880.0,24135840.0,12757680.0,23948520.0,26148480.0,23465400.0
4,1.519139e+09,2042.395833,44.158203,5592.539063,0.073105,0.022721,480.0,On,Normal,0.0,...,0.0,0.0,0.0,0.0,26528760.0,24135720.0,12757560.0,23948400.0,26148360.0,23465280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301368,1.582211e+09,1727.500000,40.437500,7743.125000,0.121094,0.021367,37553160.0,On,Normal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301369,1.582211e+09,1727.656250,40.257812,7718.242188,0.111328,0.021367,37553280.0,On,Normal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301370,1.582211e+09,1728.593750,40.375000,7781.210938,0.105977,0.024687,37553400.0,On,Normal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301371,1.582211e+09,1726.093750,40.171875,7798.476563,0.111094,0.024687,37553520.0,On,Normal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
labeledplustt.to_excel('Polished_Shit.xlsx')

In [ ]:
#for mode in allfailmodes:
mode = allfailmodes[0]
m = faildata[faildata['Mode'] == mode]
for failureinst in m['Date']:
    labeled.loc[labeled['Time']]

In [ ]:
a['Date'][4]